### Program 3:
#### Objective:
Write a program using the PyTorch framework to highlight the use of BatchNormalization and
Dropout Regularization techniques in CNNs on the CIFAR10 image dataset.
Perform the following steps:
- Preprocess data
- Define CNN architecture with & without the use of BatchNormalization and Dropout
- Define model train function
- Train both CNNs using suitable criterion and optimizer



In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Subset
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# Data preprocessing and loading
transform = transforms.Compose([
transforms.ToTensor(),
transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                                download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                            download=True, transform=transform)
train_subset = Subset(train_dataset, range(200))
test_subset = Subset(test_dataset, range(50))
train_loader = DataLoader(train_subset, batch_size=10, shuffle=True)
test_loader = DataLoader(test_subset, batch_size=10, shuffle=False)


100.0%


Extracting ./data\cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [5]:
class CNNWithBNDropout(nn.Module):
    def __init__(self):
        super(CNNWithBNDropout, self).__init__()
        self.conv_block1 = nn.Sequential(
        nn.Conv2d(3, 32, kernel_size=3, padding=1),
        nn.BatchNorm2d(32),
        nn.ReLU(),
        nn.MaxPool2d(2)
        )
        self.conv_block2 = nn.Sequential(
        nn.Conv2d(32, 64, kernel_size=3, padding=1),
        nn.BatchNorm2d(64),
        nn.ReLU(),
        nn.MaxPool2d(2)
        )
        self.dense1 = nn.Linear(64 * 8 * 8, 512)
        self.dense2 = nn.Linear(512, 10)
        self.dropout = nn.Dropout(0.5)
        self.relu = nn.ReLU()
        self.flatten = nn.Flatten()
    def forward(self, x):
        x = self.conv_block1(x)
        x = self.conv_block2(x)
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.relu(x)
        x = self.dense2(x)
        x = self.dropout(x)
        return x


In [11]:
# Function to train and evaluate a model
def train(model, optimizer, criterion, num_epochs):
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        correct_train = 0
        total_train = 0
        for data, target in train_loader:
            output = model(data)
            loss = criterion(output, target)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            predicted = torch.argmax(output.data, dim=1)
            total_train += target.size(0)
            correct_train += (predicted == target).sum().item()
        avg_train_loss = train_loss / len(train_loader)
        train_acc = 100 * correct_train / total_train
        model.eval()
        test_loss = 0.0
        correct_test = 0
        total_test = 0
        with torch.no_grad():
            for data, target in test_loader:
                output = model(data)
                loss = criterion(output, target)
                test_loss += loss.item()
                predicted = torch.argmax(output.data, dim=1)
                total_test += target.size(0)
                correct_test += (predicted==target).sum().item()
        avg_test_loss = test_loss/len(test_loader)
        test_acc = 100 * correct_test/total_test
        print(f'Epoch: [{epoch+1}/{num_epochs}], Train Loss: {avg_train_loss:.4f},Train Accuracy: {train_acc:.4f}%, Test Loss: {avg_test_loss:.4f}, Test Accuracy: {test_acc:.4f}%')

In [8]:
model = CNNWithBNDropout()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())


In [12]:
train(model, optimizer, criterion, 30)

Epoch: [1/30], Train Loss: 0.8999,Train Accuracy: 57.5000%, Test Loss: 2.6193, Test Accuracy: 16.0000%
Epoch: [2/30], Train Loss: 0.8077,Train Accuracy: 59.5000%, Test Loss: 2.7801, Test Accuracy: 18.0000%
Epoch: [3/30], Train Loss: 0.7717,Train Accuracy: 62.0000%, Test Loss: 2.6042, Test Accuracy: 24.0000%
Epoch: [4/30], Train Loss: 0.9060,Train Accuracy: 55.5000%, Test Loss: 2.8748, Test Accuracy: 18.0000%
Epoch: [5/30], Train Loss: 0.7975,Train Accuracy: 64.5000%, Test Loss: 2.7386, Test Accuracy: 20.0000%
Epoch: [6/30], Train Loss: 0.7950,Train Accuracy: 61.0000%, Test Loss: 2.6173, Test Accuracy: 22.0000%
Epoch: [7/30], Train Loss: 0.7988,Train Accuracy: 62.0000%, Test Loss: 2.7926, Test Accuracy: 18.0000%
Epoch: [8/30], Train Loss: 0.8412,Train Accuracy: 62.0000%, Test Loss: 2.9674, Test Accuracy: 16.0000%
Epoch: [9/30], Train Loss: 0.8599,Train Accuracy: 62.5000%, Test Loss: 2.9301, Test Accuracy: 18.0000%
Epoch: [10/30], Train Loss: 0.7809,Train Accuracy: 65.0000%, Test Loss: 2